# Default Rate Sensitivity on Marketing Spend Framework - MX

The effects of changing default rates on Liang & Zu's marketing spend framework is explored.

---

In [1]:
import pandas as pd
import numpy as np
import pyltv
from dbm import DBM
import plotly
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

In [6]:
data = pd.read_csv('data/mx_data_5-5.csv')

In [7]:
# keys are ltv/mCAC and values are implied ltv/average CAC
ltv_cac = {2: 8.05, 1.5: 6.34, 1.25: 5.58, 1: 4.75}

### Study Assumptions
1. Default rate stress applied as a blanket stress across all time periods.
2. Default rate stress applied to both 7dpd and 365dpd default rates.
3. The default rate stress is simply a percentage point added to the actual default rate. **So if the 365dpd default rate is currently 6%, adding a 1% stress results in a new 365dpd default rate of 7%**.

## Impact of Default Rate on LTV

In [9]:
market='mx'
forecast_method='powerslope'
min_months=4
n_months=50
default_stress=0.01

m1 = pyltv.Model(data, market=market, fcast_method=forecast_method)
m1.forecast = m1.forecast_data(m1.data, min_months, n_months)

m2 = pyltv.Model(data, market=market, fcast_method=forecast_method, default_stress=default_stress)
m2.forecast = m2.forecast_data(m2.data, min_months, n_months)

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17



In [10]:
colors = {i: c for i, c in enumerate(plotly.colors.qualitative.Dark24)}
# add additional colors
color_size = len(colors)
for i, c in enumerate(plotly.colors.qualitative.Light24):
    colors[i+color_size] = c

fig = make_subplots(rows=1, cols=2, shared_xaxes=True,
                   subplot_titles=('Unstressed', '+1% Default Rates'))

param = 'cumulative_dcf_ltv_per_original'

# LEFT PLOT
curves1=[]
for cohort in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves1.append(output)
    
for i, cohort in enumerate(curves1):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}'), row=1, col=1)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}'), row=1, col=1)

expected = m1.ltv_expected.cumulative_ltv_per_original

fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                         line=dict(color='black', width=3), legendgroup='e'), row=1, col=1)
fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                        line=dict(color='black', width=3), legendgroup='e', showlegend=False), row=1, col=2)

# RIGHT PLOT
curves2=[]
for cohort in m2.forecast.cohort.unique():
    c_data = m2.forecast[m2.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves2.append(output)
        
for i, cohort in enumerate(curves2):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2,)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2)

fig.update_layout(title='DCF LTV')
fig.layout.xaxis.title='Month'
fig.layout.xaxis2.title='Month'
fig.update_yaxes(range=[-10, 95])
fig.update_xaxes(range=[-5,52])

fig.show()

### Standard Default Rate

In [14]:
data_50mo1 = m1.forecast[m1.forecast.index==50]
ltv1 = data_50mo1['cumulative_ltv_per_original'].median()
dcf_ltv1 = data_50mo1['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv1, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv1, 2)}')

Median DCF 50mo LTV: $56.07


### +1% Default Rate Stress

In [15]:
data_50mo2 = m2.forecast[m2.forecast.index==50]
ltv2 = data_50mo2['cumulative_ltv_per_original'].median()
dcf_ltv2 = data_50mo2['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv2, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv2, 2)}')

Median DCF 50mo LTV: $46.95


In [16]:
print('Change in DCF LTV:')
print(f'{round(100*(dcf_ltv2-dcf_ltv1)/dcf_ltv1, 2)}%')

Change in DCF LTV:
-16.27%


In [17]:
ltvs = {}
for x in np.linspace(-0.03, 0.03, 7):
    m = pyltv.Model(data, market=market, fcast_method=forecast_method, default_stress=x)
    m.forecast = m.forecast_data(m.data, min_months, n_months)
    
    data_50mo = m.forecast[m.forecast.index==50]
    m.forecast['defaulted_origination'] = m.forecast['default_rate_365dpd']*m.forecast.origination_per_original
    default = m.forecast.defaulted_origination.sum()/m.forecast.origination_per_original.sum()
    
    ltvs[round(x, 2)] = (round(data_50mo['cumulative_dcf_ltv_per_original'].median(), 2), 
                         round(100*default, 2))

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17

Raw data spans 2020-09 to 2022-01
Total # of cohorts: 17



### Default Rate Stress: -3% to +3%

The plot below shows the relationship between a change in the default rate stress and the median DCF LTV for all cohorts.

In [18]:
v = [i[0] for i in ltvs.values()]
l = [f'{i[1]}%' for i in ltvs.values()]
fig = go.Figure(go.Scatter(x=l, y=v, marker=dict(size=10)))
fig.update_layout(xaxis=dict(title='Default Rate Stress'),
                 yaxis=dict(title='Weighted Average DCF LTV'))
fig.show()

In [19]:
print('For every +1% increase in default rate...')
print(f'-${round(ltvs[0][0]-ltvs[0.01][0], 2)}')

For every +1% increase in default rate...
-$9.12


The relationships between LTV and default rates is shown below and explains the linear trend. **Note that this analysis does not include the effects of default rate changes on retention.**

LTV ~ [revenue - (origination+revenue)*default_rate_365dpd]

revenue ~ [0.08 * default_rate_7dpd]

## Default Rate Historicals

#### Weighted Average Lifetime Default

This method uses the average lifetime default rate (365dpd) for each cohort. The average lifetime default rate is a weighted average of defaults by origination_per_original.

In [ ]:
defaults = []
default_changes = []
avg_lifetime_default = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c][['origination_per_original', 'default_rate_365dpd']]
    c_data['defaulted_origination'] = c_data['default_rate_365dpd']*c_data.origination_per_original
    c_data['defaulted_origination_change'] = (c_data.defaulted_origination-c_data.defaulted_origination.shift(1))/ \
        c_data.defaulted_origination
    
    defaults.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination))
    default_changes.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination_change))
    
    avg_lifetime_default.append(c_data.defaulted_origination.sum()/c_data.origination_per_original.sum())
    
avg_lifetime_default = pd.Series(avg_lifetime_default, index=m1.forecast.cohort.unique())

In [ ]:
avg_lifetime_default.mean()

In [ ]:
2*avg_lifetime_default.std()

In [ ]:
10.68-9.25

In [ ]:
fig=go.Figure(defaults)
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Defaulted Origination per Original'))
fig.show()

In [ ]:
avg = pd.Series(avg_lifetime_default.mean(), index=avg_lifetime_default.index)

fig = go.Figure([
    go.Scatter(name='default rate', x=avg_lifetime_default.index, y=avg_lifetime_default, mode='markers',
                              marker=dict(size=15)),
    go.Scatter(name='average', x=avg.index, y=avg, mode='lines')
])

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'), width=800)
fig.show()

In MX, the default rates are much higher on average. The median weighted average default rate is 9.1%. There is some apparent seasonality in the default rates that's not seen in the other markets. **We see a maximum weighted average default rate of 10.7% for MX**.

#### Unweighted Average Default

In this method, we simply look at aggregate stats of the 365 dpd default rate, without taking into account origination. This scenario looks at a blanket default rate increase across the board, regardless of the size of loans.

In [ ]:
default_stats = {}
for i, c in enumerate(m1.forecast.cohort.unique()):
    c_data = m1.forecast[m1.forecast.cohort==c][['default_rate_365dpd']]
    
    default_stats[c] = (float(c_data.median()), float(c_data.quantile(0.9)))

In [ ]:
medians=[]
quantile90=[]
for c in default_stats:
    medians.append(default_stats[c][0])
    quantile90.append(default_stats[c][1])
    
medians = pd.Series(medians, index=np.arange(1, len(medians)+1))
#params1, covs1 = curve_fit(reg, xdata=medians.index, ydata=medians)

quantile90 = pd.Series(quantile90, index=np.arange(1, len(medians)+1))
#params2, covs2 = curve_fit(reg, xdata=quantile90.index, ydata=quantile90)
    
fig = go.Figure([
    go.Scatter(name='median default', x=medians.index, y=medians, mode='markers'),
    #go.Scatter(name='median-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params1[0], params1[1])),
    go.Scatter(name='90 percentile', x=quantile90.index, y=quantile90, mode='markers')
    #go.Scatter(name='90 percentile-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params2[0], params2[1]))
])

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

Similarly, in looking at both the 50% and 90% quantiles of the 365dpd default rates, the trend is quite flat with some similar seasonality as the weighted average default rates. The max we see here for MX is 12.45%.

## Payback Period

### mCAC Payback Period

In [ ]:
ratios = [1.25, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        
        avg_cac = float(dcf_ltv/ltv_cac[r])

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [ ]:
ratios = [1.25, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        
        avg_cac = float(dcf_ltv/ltv_cac[r])
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The table below shows the payback periods for unstressed default rates in the 1.5x scenario.

In [ ]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

The second table shows the new payback periods in the same 1.5x scenario, but with a +1% default rate stress applied. The payback periods are pushed out by several months.

In [ ]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

In [ ]:
fig1 = go.Figure([go.Scatter(name='1.25x', x=payback_results1['Payback Period (1.25)'], 
                            y=payback_results1['Payback Amount (1.25)'], mode='markers',
                            marker=dict(color='red')),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[12, 40]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1.25x', x=payback_results2['Payback Period (1.25)'], 
                            y=payback_results2['Payback Amount (1.25)'], mode='markers',
                            marker=dict(color='red')),
                  go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[12, 40]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [ ]:
payback_results1

In [ ]:
payback_results2

In [ ]:
payback_results1['Payback Period (1.25)'].mean()

In [ ]:
for r in ratios:
    print(f'Ratio... {r}')
    print(f"Unstressed: {round(payback_results1[f'Payback Period ({r})'].mean())}")
    print(f"+1% Stress: {round(payback_results2[f'Payback Period ({r})'].mean())}")

In the worst case, the **payback period will shift out 11 months at an LTV/mCAC of 1.5**, and by **6 months for an LTV/mCAC of 2**.

### Average CAC Payback Period

In [ ]:
ratios = [1.25, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        
        avg_cac = float(dcf_ltv/ltv_cac[r])

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [ ]:
ratios = [1.25, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        
        avg_cac = float(dcf_ltv/ltv_cac[r])
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The two tables below show the changes in payback periods when using LTV/CAC instead of LTV/mCAC as the guard rail. The shifts in payback periods are much less significant.

In [ ]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

In [ ]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

In [ ]:
fig1 = go.Figure([go.Scatter(name='1.25x', x=payback_results1['Payback Period (1.25)'], 
                            y=payback_results1['Payback Amount (1.25)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers')
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[5, 23]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1.25x', x=payback_results2['Payback Period (1.25)'], 
                            y=payback_results2['Payback Amount (1.25)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers')
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[5, 23]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [ ]:
payback_results1

In [ ]:
payback_results2

In [ ]:
payback_results1['Payback Period (2)'].mean()

In [ ]:
for r in ratios:
    print(f'Ratio... {r}')
    print(f"Unstressed: {round(payback_results1[f'Payback Period ({r})'].mean())}")
    print(f"+1% Stress: {round(payback_results2[f'Payback Period ({r})'].mean())}")

In the worst case, our most recent cohort, the **payback period will shift out by 4 months at an LTV/CAC of 1.5x**, and by **3 for an LTV/CAC of 2x**.

## Conclusions

- A rise in default rate leads to an increas in payback period. This is true for any scenario (1x, 1.5x, 2x). In general, the shift in payback period is worse for lower multipliers (less margin).
- A +1% increase in default rate in the 1.5x scenario will shift the payback period by up to 11 months. At 2x, there is only a shift in payback period of +2 months.
- With a 1.5x scenario, the data suggests we can still maintain a 50mo positive LTV inspite of up to a +1% rise in default rates.